# Team 2 Capstone Flight Matching

3/1/21 Progress

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
d_iata = 'lax'
a_iata = 'las'
airline_icao = 'dal'
date_from = '2020-11-08'
date_to = '2020-11-09'
op_name_icao = {'delta air lines':'dal', 
                'delta airlines':'dal',
                'united airlines':'ual',
               'southwest airlines':'swa', 
               'spirit airlines':'nks', 
               'alaska airlines':'asa', 
               'american airlines':'aal'}


## ADS-B Preprocessing

In [3]:
# flight = pd.read_csv('test.csv')

flight = pd.read_parquet('s3://capstone-testing-flights/flights/2020-11-09/')
flight = flight[['Icao', 'DepartureAirport', 'ArrivalAirport', 
                'StartTime', 'LiftOffTime', 'TouchDownTime',
                'StopTime', 'Op']]

In [4]:
flight.DepartureAirport = flight.DepartureAirport.map(str.lower)
flight.ArrivalAirport = flight.ArrivalAirport.map(str.lower)
flight.Op = flight.Op.map(str.lower)

In [5]:
flight = flight[(flight.DepartureAirport==d_iata) & (flight.ArrivalAirport==a_iata)]

In [6]:
flight = flight[flight.Op.isin(op_name_icao.keys())]

In [7]:
flight['Op_icao'] = flight.Op.map(op_name_icao)

In [8]:
flight.LiftOffTime =pd.to_datetime(flight.LiftOffTime)
flight.TouchDownTime =pd.to_datetime(flight.TouchDownTime)

In [9]:
flight.StartTime = flight.StartTime - datetime.timedelta(hours=8)
flight.LiftOffTime = flight.LiftOffTime - datetime.timedelta(hours=8)
flight.TouchDownTime = flight.TouchDownTime - datetime.timedelta(hours=8)
flight.StopTime = flight.StopTime - datetime.timedelta(hours=8)

In [10]:
flight['c_key'] = flight.Op_icao + '_' + flight.DepartureAirport + '_' + flight.ArrivalAirport + '_' + flight.LiftOffTime.dt.floor('H').dt.strftime('%Y-%m-%d:%H')
#+ '_' + flight.TouchDownTime.round('30min').dt.strftime('%Y-%m-%d:%H')

In [11]:
flight['c_key1'] = flight.Op_icao + '_' + flight.DepartureAirport + '_' + flight.ArrivalAirport + '_' + flight.LiftOffTime.round('H').dt.strftime('%Y-%m-%d:%H')

In [12]:
flight

,Icao,DepartureAirport,ArrivalAirport,StartTime,LiftOffTime,TouchDownTime,StopTime,Op,Op_icao,c_key,c_key1
1507,A4794E,lax,las,2020-11-08 20:03:43.317,2020-11-08 21:24:44.484,2020-11-08 22:11:15.197,2020-11-08 22:15:13.593,delta air lines,dal,dal_lax_las_2020-11-08:21,dal_lax_las_2020-11-08:21
1642,A737C7,lax,las,2020-11-08 18:21:44.641,2020-11-08 19:02:58.712,2020-11-08 19:49:58.648,2020-11-08 19:55:14.255,southwest airlines,swa,swa_lax_las_2020-11-08:19,swa_lax_las_2020-11-08:19
4047,ACA788,lax,las,2020-11-08 17:17:24.484,2020-11-08 17:41:15.350,2020-11-08 18:25:45.661,2020-11-08 19:16:28.667,southwest airlines,swa,swa_lax_las_2020-11-08:17,swa_lax_las_2020-11-08:18
4341,ACE5BC,lax,las,2020-11-08 17:37:59.339,2020-11-09 11:57:30.075,2020-11-09 12:38:14.434,2020-11-09 12:46:58.649,american airlines,aal,aal_lax_las_2020-11-09:11,aal_lax_las_2020-11-09:12
5166,A81B2A,lax,las,2020-11-08 19:52:24.348,2020-11-08 20:03:58.990,2020-11-08 20:48:14.777,2020-11-08 20:49:43.867,spirit airlines,nks,nks_lax_las_2020-11-08:20,nks_lax_las_2020-11-08:20
5850,ABF2EA,lax,las,2020-11-09 09:03:58.292,2020-11-09 09:50:28.769,2020-11-09 10:38:45.766,2020-11-09 11:21:10.996,southwest airlines,swa,swa_lax_las_2020-11-09:09,swa_lax_las_2020-11-09:10
6221,AA6ADE,lax,las,2020-11-09 06:37:36.525,2020-11-09 10:16:28.875,2020-11-09 11:02:28.363,2020-11-09 11:10:43.603,american airlines,aal,aal_lax_las_2020-11-09:10,aal_lax_las_2020-11-09:10
7934,AC8E1A,lax,las,2020-11-08 18:18:59.848,2020-11-08 19:10:14.456,2020-11-09 09:59:29.508,2020-11-09 11:05:58.322,spirit airlines,nks,nks_lax_las_2020-11-08:19,nks_lax_las_2020-11-08:19
8010,A6C120,lax,las,2020-11-08 17:24:45.401,2020-11-08 18:41:59.139,2020-11-08 19:26:28.032,2020-11-08 19:38:26.290,american airlines,aal,aal_lax_las_2020-11-08:18,aal_lax_las_2020-11-08:19
8378,A2685B,lax,las,2020-11-08 20:42:28.476,2020-11-09 08:35:00.700,2020-11-09 09:16:44.914,2020-11-09 10:43:14.854,southwest airlines,swa,swa_lax_las_2020-11-09:08,swa_lax_las_2020-11-09:09


In [25]:
flight.to_csv('flight.csv')

## Historical Schedule Preprocessing

In [13]:
import requests as re

s = 'https://aviation-edge.com/v2/public/flightsHistory?key={}&code={}&&type=departure&date_from={}&date_to={}'

data = re.get(s.format(key, d_iata, date_from, date_to)).json()
df = pd.json_normalize(data)
df = df[df['arrival.iataCode']==a_iata]
df = df[df['airline.icaoCode'].isin(op_name_icao.values())]
df['departure.scheduledTime'] =pd.to_datetime(df['departure.scheduledTime'])
df['arrival.scheduledTime'] =pd.to_datetime(df['arrival.scheduledTime'])
df = df[['departure.iataCode','departure.scheduledTime', 'departure.actualTime', 'arrival.iataCode', 'arrival.scheduledTime', 'arrival.estimatedTime', 'airline.icaoCode', 'airline.name']]


In [14]:
df['c_key'] = df['airline.icaoCode'] + '_' + df['departure.iataCode'] + '_' + df['arrival.iataCode'] + '_' + df['departure.scheduledTime'].dt.floor('H').dt.strftime('%Y-%m-%d:%H') 
#+ '_' + df['arrival.scheduledTime'].round('30min').dt.strftime('%Y-%m-%d:%H')

In [15]:
df['c_key1'] = df['airline.icaoCode'] + '_' + df['departure.iataCode'] + '_' + df['arrival.iataCode'] + '_' + df['departure.scheduledTime'].round('H').dt.strftime('%Y-%m-%d:%H') 

In [16]:
df

,departure.iataCode,departure.scheduledTime,departure.actualTime,arrival.iataCode,arrival.scheduledTime,arrival.estimatedTime,airline.icaoCode,airline.name,c_key,c_key1
83,lax,2020-11-07 18:32:00,NaN,las,2020-11-07 19:40:00,NaN,aal,american airlines,aal_lax_las_2020-11-07:18,aal_lax_las_2020-11-07:19
151,lax,2020-11-07 19:59:00,2020-11-07t20:21:00.000,las,2020-11-07 21:10:00,2020-11-07t21:05:00.000,nks,spirit airlines,nks_lax_las_2020-11-07:19,nks_lax_las_2020-11-07:20
223,lax,2020-11-07 21:20:00,2020-11-07t21:27:00.000,las,2020-11-07 22:30:00,2020-11-07t22:10:00.000,dal,delta air lines,dal_lax_las_2020-11-07:21,dal_lax_las_2020-11-07:21
265,lax,2020-11-07 22:35:00,2020-11-07t22:51:00.000,las,2020-11-07 23:53:00,2020-11-07t23:32:00.000,ual,united airlines,ual_lax_las_2020-11-07:22,ual_lax_las_2020-11-07:23
434,lax,2020-11-08 07:10:00,2020-11-08t07:13:00.000,las,2020-11-08 08:20:00,2020-11-08t07:55:00.000,dal,delta air lines,dal_lax_las_2020-11-08:07,dal_lax_las_2020-11-08:07
439,lax,2020-11-08 07:25:00,2020-11-08t07:37:00.000,las,2020-11-08 08:35:00,2020-11-08t08:21:00.000,swa,southwest airlines,swa_lax_las_2020-11-08:07,swa_lax_las_2020-11-08:07
495,lax,2020-11-08 08:20:00,NaN,las,2020-11-08 09:40:00,NaN,ual,united airlines,ual_lax_las_2020-11-08:08,ual_lax_las_2020-11-08:08
554,lax,2020-11-08 09:00:00,2020-11-08t09:00:00.000,las,2020-11-08 10:13:00,2020-11-08t09:48:00.000,aal,american airlines,aal_lax_las_2020-11-08:09,aal_lax_las_2020-11-08:09
644,lax,2020-11-08 09:50:00,2020-11-08t09:58:00.000,las,2020-11-08 11:05:00,2020-11-08t10:44:00.000,swa,southwest airlines,swa_lax_las_2020-11-08:09,swa_lax_las_2020-11-08:10
700,lax,2020-11-08 10:45:00,2020-11-08t10:53:00.000,las,2020-11-08 12:02:00,2020-11-08t11:39:00.000,dal,delta air lines,dal_lax_las_2020-11-08:10,dal_lax_las_2020-11-08:11


## Joining

In [17]:
merged = flight.merge(df, on='c_key')

In [18]:
merged

,Icao,DepartureAirport,ArrivalAirport,StartTime,LiftOffTime,TouchDownTime,StopTime,Op,Op_icao,c_key,c_key1_x,departure.iataCode,departure.scheduledTime,departure.actualTime,arrival.iataCode,arrival.scheduledTime,arrival.estimatedTime,airline.icaoCode,airline.name,c_key1_y
0,A4794E,lax,las,2020-11-08 20:03:43.317,2020-11-08 21:24:44.484,2020-11-08 22:11:15.197,2020-11-08 22:15:13.593,delta air lines,dal,dal_lax_las_2020-11-08:21,dal_lax_las_2020-11-08:21,lax,2020-11-08 21:20:00,2020-11-08t21:24:00.000,las,2020-11-08 22:25:00,2020-11-08t22:06:00.000,dal,delta air lines,dal_lax_las_2020-11-08:21
1,A737C7,lax,las,2020-11-08 18:21:44.641,2020-11-08 19:02:58.712,2020-11-08 19:49:58.648,2020-11-08 19:55:14.255,southwest airlines,swa,swa_lax_las_2020-11-08:19,swa_lax_las_2020-11-08:19,lax,2020-11-08 19:00:00,2020-11-08t19:02:00.000,las,2020-11-08 20:10:00,2020-11-08t19:56:00.000,swa,southwest airlines,swa_lax_las_2020-11-08:19
2,ACA788,lax,las,2020-11-08 17:17:24.484,2020-11-08 17:41:15.350,2020-11-08 18:25:45.661,2020-11-08 19:16:28.667,southwest airlines,swa,swa_lax_las_2020-11-08:17,swa_lax_las_2020-11-08:18,lax,2020-11-08 17:35:00,2020-11-08t17:41:00.000,las,2020-11-08 18:45:00,2020-11-08t18:22:00.000,swa,southwest airlines,swa_lax_las_2020-11-08:18
3,ACE5BC,lax,las,2020-11-08 17:37:59.339,2020-11-09 11:57:30.075,2020-11-09 12:38:14.434,2020-11-09 12:46:58.649,american airlines,aal,aal_lax_las_2020-11-09:11,aal_lax_las_2020-11-09:12,lax,2020-11-09 11:50:00,2020-11-09t11:57:00.000,las,2020-11-09 13:00:00,2020-11-09t12:43:00.000,aal,american airlines,aal_lax_las_2020-11-09:12
4,ABF2EA,lax,las,2020-11-09 09:03:58.292,2020-11-09 09:50:28.769,2020-11-09 10:38:45.766,2020-11-09 11:21:10.996,southwest airlines,swa,swa_lax_las_2020-11-09:09,swa_lax_las_2020-11-09:10,lax,2020-11-09 09:50:00,2020-11-09t09:50:00.000,las,2020-11-09 11:05:00,2020-11-09t10:42:00.000,swa,southwest airlines,swa_lax_las_2020-11-09:10
5,AC8E1A,lax,las,2020-11-08 18:18:59.848,2020-11-08 19:10:14.456,2020-11-09 09:59:29.508,2020-11-09 11:05:58.322,spirit airlines,nks,nks_lax_las_2020-11-08:19,nks_lax_las_2020-11-08:19,lax,2020-11-08 19:59:00,2020-11-08t20:03:00.000,las,2020-11-08 21:10:00,2020-11-08t20:47:00.000,nks,spirit airlines,nks_lax_las_2020-11-08:20
6,A6C120,lax,las,2020-11-08 17:24:45.401,2020-11-08 18:41:59.139,2020-11-08 19:26:28.032,2020-11-08 19:38:26.290,american airlines,aal,aal_lax_las_2020-11-08:18,aal_lax_las_2020-11-08:19,lax,2020-11-08 18:32:00,2020-11-08t18:41:00.000,las,2020-11-08 19:40:00,2020-11-08t19:29:00.000,aal,american airlines,aal_lax_las_2020-11-08:19
7,A2685B,lax,las,2020-11-08 20:42:28.476,2020-11-09 08:35:00.700,2020-11-09 09:16:44.914,2020-11-09 10:43:14.854,southwest airlines,swa,swa_lax_las_2020-11-09:08,swa_lax_las_2020-11-09:09,lax,2020-11-09 08:30:00,NaN,las,2020-11-09 09:50:00,NaN,swa,southwest airlines,swa_lax_las_2020-11-09:08


In [19]:
merged2 = flight.merge(df, on='c_key1')

In [20]:
merged2

,Icao,DepartureAirport,ArrivalAirport,StartTime,LiftOffTime,TouchDownTime,StopTime,Op,Op_icao,c_key_x,c_key1,departure.iataCode,departure.scheduledTime,departure.actualTime,arrival.iataCode,arrival.scheduledTime,arrival.estimatedTime,airline.icaoCode,airline.name,c_key_y
0,A4794E,lax,las,2020-11-08 20:03:43.317,2020-11-08 21:24:44.484,2020-11-08 22:11:15.197,2020-11-08 22:15:13.593,delta air lines,dal,dal_lax_las_2020-11-08:21,dal_lax_las_2020-11-08:21,lax,2020-11-08 21:20:00,2020-11-08t21:24:00.000,las,2020-11-08 22:25:00,2020-11-08t22:06:00.000,dal,delta air lines,dal_lax_las_2020-11-08:21
1,A737C7,lax,las,2020-11-08 18:21:44.641,2020-11-08 19:02:58.712,2020-11-08 19:49:58.648,2020-11-08 19:55:14.255,southwest airlines,swa,swa_lax_las_2020-11-08:19,swa_lax_las_2020-11-08:19,lax,2020-11-08 19:00:00,2020-11-08t19:02:00.000,las,2020-11-08 20:10:00,2020-11-08t19:56:00.000,swa,southwest airlines,swa_lax_las_2020-11-08:19
2,ACA788,lax,las,2020-11-08 17:17:24.484,2020-11-08 17:41:15.350,2020-11-08 18:25:45.661,2020-11-08 19:16:28.667,southwest airlines,swa,swa_lax_las_2020-11-08:17,swa_lax_las_2020-11-08:18,lax,2020-11-08 17:35:00,2020-11-08t17:41:00.000,las,2020-11-08 18:45:00,2020-11-08t18:22:00.000,swa,southwest airlines,swa_lax_las_2020-11-08:17
3,ACE5BC,lax,las,2020-11-08 17:37:59.339,2020-11-09 11:57:30.075,2020-11-09 12:38:14.434,2020-11-09 12:46:58.649,american airlines,aal,aal_lax_las_2020-11-09:11,aal_lax_las_2020-11-09:12,lax,2020-11-09 11:50:00,2020-11-09t11:57:00.000,las,2020-11-09 13:00:00,2020-11-09t12:43:00.000,aal,american airlines,aal_lax_las_2020-11-09:11
4,A81B2A,lax,las,2020-11-08 19:52:24.348,2020-11-08 20:03:58.990,2020-11-08 20:48:14.777,2020-11-08 20:49:43.867,spirit airlines,nks,nks_lax_las_2020-11-08:20,nks_lax_las_2020-11-08:20,lax,2020-11-08 19:59:00,2020-11-08t20:03:00.000,las,2020-11-08 21:10:00,2020-11-08t20:47:00.000,nks,spirit airlines,nks_lax_las_2020-11-08:19
5,ABF2EA,lax,las,2020-11-09 09:03:58.292,2020-11-09 09:50:28.769,2020-11-09 10:38:45.766,2020-11-09 11:21:10.996,southwest airlines,swa,swa_lax_las_2020-11-09:09,swa_lax_las_2020-11-09:10,lax,2020-11-09 09:50:00,2020-11-09t09:50:00.000,las,2020-11-09 11:05:00,2020-11-09t10:42:00.000,swa,southwest airlines,swa_lax_las_2020-11-09:09
6,A6C120,lax,las,2020-11-08 17:24:45.401,2020-11-08 18:41:59.139,2020-11-08 19:26:28.032,2020-11-08 19:38:26.290,american airlines,aal,aal_lax_las_2020-11-08:18,aal_lax_las_2020-11-08:19,lax,2020-11-08 18:32:00,2020-11-08t18:41:00.000,las,2020-11-08 19:40:00,2020-11-08t19:29:00.000,aal,american airlines,aal_lax_las_2020-11-08:18


In [21]:
merged2.columns

Index(['Icao', 'DepartureAirport', 'ArrivalAirport', 'StartTime',
       'LiftOffTime', 'TouchDownTime', 'StopTime', 'Op', 'Op_icao', 'c_key_x',
       'c_key1', 'departure.iataCode', 'departure.scheduledTime',
       'departure.actualTime', 'arrival.iataCode', 'arrival.scheduledTime',
       'arrival.estimatedTime', 'airline.icaoCode', 'airline.name', 'c_key_y'],
      dtype='object')

In [22]:
result = pd.concat([merged, merged2]).drop_duplicates(['Icao', 'DepartureAirport', 'ArrivalAirport', 'StartTime',
       'LiftOffTime'])
result = result[['Icao', 'Op', 'Op_icao', 'DepartureAirport', 'ArrivalAirport', 'StartTime', 'LiftOffTime','departure.scheduledTime', 'departure.actualTime', 'TouchDownTime', 'StopTime', 'arrival.scheduledTime', 'arrival.estimatedTime']]

In [23]:
result

,Icao,Op,Op_icao,DepartureAirport,ArrivalAirport,StartTime,LiftOffTime,departure.scheduledTime,departure.actualTime,TouchDownTime,StopTime,arrival.scheduledTime,arrival.estimatedTime
0,A4794E,delta air lines,dal,lax,las,2020-11-08 20:03:43.317,2020-11-08 21:24:44.484,2020-11-08 21:20:00,2020-11-08t21:24:00.000,2020-11-08 22:11:15.197,2020-11-08 22:15:13.593,2020-11-08 22:25:00,2020-11-08t22:06:00.000
1,A737C7,southwest airlines,swa,lax,las,2020-11-08 18:21:44.641,2020-11-08 19:02:58.712,2020-11-08 19:00:00,2020-11-08t19:02:00.000,2020-11-08 19:49:58.648,2020-11-08 19:55:14.255,2020-11-08 20:10:00,2020-11-08t19:56:00.000
2,ACA788,southwest airlines,swa,lax,las,2020-11-08 17:17:24.484,2020-11-08 17:41:15.350,2020-11-08 17:35:00,2020-11-08t17:41:00.000,2020-11-08 18:25:45.661,2020-11-08 19:16:28.667,2020-11-08 18:45:00,2020-11-08t18:22:00.000
3,ACE5BC,american airlines,aal,lax,las,2020-11-08 17:37:59.339,2020-11-09 11:57:30.075,2020-11-09 11:50:00,2020-11-09t11:57:00.000,2020-11-09 12:38:14.434,2020-11-09 12:46:58.649,2020-11-09 13:00:00,2020-11-09t12:43:00.000
4,ABF2EA,southwest airlines,swa,lax,las,2020-11-09 09:03:58.292,2020-11-09 09:50:28.769,2020-11-09 09:50:00,2020-11-09t09:50:00.000,2020-11-09 10:38:45.766,2020-11-09 11:21:10.996,2020-11-09 11:05:00,2020-11-09t10:42:00.000
5,AC8E1A,spirit airlines,nks,lax,las,2020-11-08 18:18:59.848,2020-11-08 19:10:14.456,2020-11-08 19:59:00,2020-11-08t20:03:00.000,2020-11-09 09:59:29.508,2020-11-09 11:05:58.322,2020-11-08 21:10:00,2020-11-08t20:47:00.000
6,A6C120,american airlines,aal,lax,las,2020-11-08 17:24:45.401,2020-11-08 18:41:59.139,2020-11-08 18:32:00,2020-11-08t18:41:00.000,2020-11-08 19:26:28.032,2020-11-08 19:38:26.290,2020-11-08 19:40:00,2020-11-08t19:29:00.000
7,A2685B,southwest airlines,swa,lax,las,2020-11-08 20:42:28.476,2020-11-09 08:35:00.700,2020-11-09 08:30:00,NaN,2020-11-09 09:16:44.914,2020-11-09 10:43:14.854,2020-11-09 09:50:00,NaN
4,A81B2A,spirit airlines,nks,lax,las,2020-11-08 19:52:24.348,2020-11-08 20:03:58.990,2020-11-08 19:59:00,2020-11-08t20:03:00.000,2020-11-08 20:48:14.777,2020-11-08 20:49:43.867,2020-11-08 21:10:00,2020-11-08t20:47:00.000
